#### Author: Kubam Ivo
#### Date: 7/23/2020

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

In [ ]:
# Create and train model
import joblib
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge

dataset_x, dataset_y = load_diabetes(return_X_y=True)

model = Ridge().fit(dataset_x, dataset_y)

joblib.dump(model, 'sklearn_regression_model.pkl')

In [ ]:
# Register the input and output datasets
import numpy as np
from azureml.core import Dataset

np.savetxt('features.csv', dataset_x,delimiter=',')
np.savetxt('labels.csv', dataset_y, delimiter=',')

datastore = ws.get_default_datastore()
datastore.upload_files(files= ['./features.csv', './labels.csv'], target_path = 'sklearn_regression/',overwrite=True)

input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore,'sklearn_regression/features.csv')])
output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/labels.csv')])

In [ ]:
#Model registration
import sklearn
from azureml.core.model  import Model
from azureml.core.resource_configuration import ResourceConfiguration
model = Model.register(workspace= ws,
        model_path= "./sklearn_regression_model.pkl", #local file to upload
        model_name = 'my_sklearn-model', # Name of the registered model in workspace
        model_framework = Model.Framework.SCIKITLEARN,
        model_framework_version=sklearn.__version__,
        sample_input_dataset = input_dataset,
        sample_output_dataset= output_dataset,
        resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
        tags= {'area': 'diabestes','type':'regression' },
        description= 'Ridge regression model to predict diabetes progression.')

print('name: ', model.name)
print('version: ', model.version)

In [ ]:
# Deploy Model using default environment
service_name = 'my-sklearn-service'

service = model.deploy(ws, service_name,[model],overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
# Deploy model using custom environment
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
        'azureml-defaults', 
        'inference-schema[numpy-support]',
        'joblib',
        'numpy',
        'scikit-learn == {}'.format(sklearn.__version__)
])

In [ ]:

with open('score.py') as f:
    print(f.read())

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'my-custom-env-service'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)